# 🏠🏠🏠 Projet Kaggle : Présentation et storytelling 🏠🏠🏠

## Initialisation

### Importation des bibliothèques nécessaires


In [1]:
import json

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from dash import Dash, dcc, html
from pingouin import kruskal
from plotly.colors import n_colors
from scipy.stats import chi2_contingency

### Lecture du jeux de données d'entrainement


In [2]:
with open("../data/processed/dtype_dict.json") as f:
    dtype_dict = json.load(f)

train = pd.read_csv(
    "../data/processed/train.csv",
    delimiter=",",
    encoding="utf-8",
    index_col="Id",
    dtype=dtype_dict,
)

### Définition d'un thème général


In [3]:
# Template personnalisé
monTheme = go.layout.Template(
    layout=dict(
        template="simple_white",
        autosize=True,
        font=dict(family="Arial", size=15, color="#000000"),
        title=dict(font=dict(size=35, family="Arial"), x=0.5),
        xaxis=dict(tickangle=-35, automargin=True),
        yaxis=dict(tickangle=-35, automargin=True),
    )
)

### Enregistrement des principales propriétés


In [4]:
# Enregistrement du template
pio.templates["monTheme"] = monTheme

# Définition du template comme template par défaut
pio.templates.default = "monTheme"

### Définition d'un style générique pour des boutons


In [5]:
# Même principe, style de boutons par defaut
# Ne peut pas rentrer dans les templates
styleBoutons = dict(
    bgcolor="#6B6B6B",
    bordercolor="#000000",
    borderwidth=1.5,
    direction="right",
    font_weight=700,
    showactive=True,
    type="buttons",
    x=1,
    xanchor="right",
    y=1.2,
    yanchor="top",
)

### Définition d'un style générique pour un curseur


In [6]:
# Même chose pour le curseur
styleSlider = dict(
    active=0,
    font_color="rgba(0,0,0,0)",
    len=0.2,  # Longueur du slider
    lenmode="fraction",  # Mode de longueur du slider
    tickcolor="rgba(0,0,0,0)",
    x=0,  # Position x du slider
    xanchor="left",
    y=1.35,  # Position y du slider
    yanchor="top",
)

### Définition d'un dictionnaire de style pour les polices (Dash)


In [7]:
mesPolices = {
    "font-size": 25,
    "font-family": "Arial",
    "font-weight": 700,
    "color": "Black",
}

### Définition de dictionnaires pour le style des cellules du tableau html (Dash)


In [8]:
# style des bordures
border = "1px solid black"
center_text = "center"
padding = "5px"

# Définir le style pour les cellules d'en-tête
header_style = {
    "text-align": center_text,
    "background-color": "lightblue",
    "padding": padding,
    "border": border,
}

# Définir le style pour les cellules de corps
cell_style = {"text-align": center_text, "padding": padding, "border": border}

## Quelle est la répartition du prix des maisons ?


In [9]:
price = px.histogram(
    train, x="SalePrice", marginal="box", color_discrete_sequence=["rgb(200, 10, 10)"]
)

price.update_layout(
    title_text="Répartition du prix des maisons",
    xaxis_title="Prix des maisons",
    yaxis_title="Nombre",
    showlegend=False,
)

price.show()

## Quelle est la répartition de la taille des maisons ?


In [10]:
colors = n_colors(
    "rgb(5, 200, 200)",
    "rgb(200, 10, 10)",
    5,
    colortype="rgb",
)

size = px.histogram(
    train,
    x=["1stFlrSF", "2ndFlrSF", "TotalBsmtSF", "GrLivArea", "GarageArea"],
    marginal="box",
    color_discrete_sequence=colors,
)

custom_labels = {
    "1stFlrSF": "Surface du rez de chaussée",
    "2ndFlrSF": "Surface de l'étage",
    "TotalBsmtSF": "Surface du sous-sol",
    "GrLivArea": "Surface globale",
    "GarageArea": "Surface du garage",
}

size.update_layout(
    title_text="Répartition de la taille des maisons, avec les différentes surfaces",
    xaxis_title="Taille des surfaces en SF",
    yaxis_title="Nombre",
    barmode="stack",
    legend_title_text="Surfaces en SF",
)

size.for_each_trace(
    lambda t: t.update(name=custom_labels[t.name]) if t.name in custom_labels else ()
)
size.show()

## A quoi ressemble le top 3 des maisons les plus chères?


In [11]:
train[
    [
        "SalePrice",
        "1stFlrSF",
        "2ndFlrSF",
        "TotalBsmtSF",
        "KitchenQual",
        "ExterQual",
        "Fireplaces",
        "Neighborhood",
        "MiscFeature",
        "OverallQual",
        "FullBath",
    ]
].sort_values("SalePrice", ascending=False).head(3)

,SalePrice,1stFlrSF,2ndFlrSF,TotalBsmtSF,KitchenQual,ExterQual,Fireplaces,Neighborhood,MiscFeature,OverallQual,FullBath
Id,,,,,,,,,,,
692,755000,2444,1872,2444,Excellent,Excellent,2,Northridge,No specificity,Very Excellent,3
1183,745000,2411,2065,2396,Excellent,Good,2,Northridge,No specificity,Very Excellent,3
1170,625000,1831,1796,1930,Good,Good,1,Northridge,No specificity,Very Excellent,3


## Est-ce qu'il y a un essoufflement de l'immobilier, au fur et à mesure des années, pour Ames?


In [12]:
train[["SalePrice", "YrSold"]].groupby(
    "YrSold", dropna=False, observed=False
).describe()

SalePrice                                                            \
           count           mean           std      min       25%       50%   
YrSold                                                                       
2006       314.0  182549.458599  79426.838855  35311.0  131375.0  163995.0   
2007       329.0  186063.151976  85768.171410  39300.0  129900.0  167000.0   
2008       304.0  177360.838816  69735.610685  40000.0  131250.0  164000.0   
2009       338.0  179432.103550  80879.237311  34900.0  125250.0  162000.0   
2010       175.0  177393.674286  80451.280085  55000.0  128100.0  155000.0   

                            
             75%       max  
YrSold                      
2006    218782.5  625000.0  
2007    219500.0  755000.0  
2008    207000.0  446261.0  
2009    212750.0  582933.0  
2010    213250.0  611657.0

In [13]:
kruskal(data=train[["SalePrice", "YrSold"]], dv="SalePrice", between="YrSold")

,Source,ddof1,H,p-unc
Kruskal,YrSold,4,1.645903,0.800519


In [14]:
# Définition des couleurs
colors = n_colors(
    "rgb(5, 200, 200)",
    "rgb(200, 10, 10)",
    train["YrSold"].nunique(dropna=False),
    colortype="rgb",
)

# Calculer la médiane de la variable quantitative pour chaque catégorie
medians = train.groupby("YrSold")["SalePrice"].median().sort_values()

# Définition de la figure
yr = go.Figure()

# Ajout des traces avec un ordre de superposition
for i, cat in enumerate(medians.index):
    yr.add_trace(
        go.Violin(
            x=train[train["YrSold"] == cat]["SalePrice"],
            name=cat,
            line_color="black",
            fillcolor=colors[i],
            width=3,
            points=False,
            opacity=0.8,
        )
    )

# Mise à jour des traces
yr.update_traces(orientation="h", side="positive", width=3, points=False, opacity=1)

# Mise à jour du layout
yr.update_layout(
    title_text="Prix de vente au fil des années",
    legend_title_text="Année de vente",
    xaxis_showgrid=False,
    xaxis_zeroline=False,
    height=900,
    xaxis_title="Prix de vente",
    yaxis_title="Année de vente",
    # yaxis_autorange="reversed",
)

yr.show()

Le test statistique ne révèle aucune significativité. Cette conclusion est également apparente visuellement : les prix de vente à Ames, sur la période considérée et en fonction de notre ensemble de données, n'ont pas connu d'évolutions notables.


## Est-ce qu'on achète plus en été?

Attention au piège : il ne faut pas séléctionner la dernière année pour appliquer le test statistique, car les données de cette année ne sont pas complètes.


In [15]:
# Tableau de contingence
contingency_table = pd.crosstab(
    train[train["YrSold"] != "2010"]["MoSold"], columns="count", dropna=False
)

# Calculer le total des ventes
total_sales = contingency_table["count"].sum()

# Calculer les fréquences attendues sous l'hypothèse d'une distribution uniforme
expected_counts = [total_sales / 12] * 12

contingency_table["unif"] = expected_counts

# Test du chi2 d'indépendance
chi2, p, dof, exp_freq = chi2_contingency(contingency_table)

p

5.040374104517819e-33

Donc il y a bien une différence du nombre de vente entre les mois.


In [16]:
summer = px.histogram(
    train,
    x="MoSold",
    color="YrSold",
    category_orders={  # replaces default order by column name
        "YrSold": ["2006", "2007", "2008", "2009", "2010"],
        "MoSold": [
            "Janvier",
            "Février",
            "Mars",
            "Avril",
            "Mai",
            "Juin",
            "Juillet",
            "Août",
            "Septembre",
            "Octobre",
            "Novembre",
            "Décembre",
        ],
    },
    color_discrete_sequence=colors,
)

# Ajouter une ligne rouge à la moyenne
summer.add_shape(
    type="line",
    # Les mois sont indexés de 0 à 11
    x0=-0.5,
    x1=11.5,
    y0=expected_counts[0],
    y1=expected_counts[0],
    line=dict(color="red", width=2),
)

# Ajouter une annotation avec une flèche
summer.add_annotation(
    x=9,
    y=expected_counts[0] + 30,
    text="Moyenne des ventes",
    showarrow=True,
    arrowhead=3,
    ax=70,
    ay=-100,
)

summer.update_layout(
    title_text="Nombre de vente par mois pour chaque année",
    legend_title_text="Année de vente",
    xaxis_title="Mois de vente",
    yaxis_title="Nombre de vente",
)

summer.show()

### Création d'une app Dash

#### Initialisation


In [17]:
# Initialisation de l'application Dash
app = Dash(__name__)

In [18]:
# Fonction utilitaire pour générer un tableau HTML à partir d'un DataFrame
def generate_table(dataframe):
    header = [html.Th(["", dataframe.columns[0]], style=header_style)] + [
        html.Th(f"{dataframe.columns.name} : {col}", style=header_style)
        for col in dataframe.columns[1:]
    ]
    rows = []

    for idx in dataframe.index:
        row = []
        row += [
            html.Td(round(val, 2), style=cell_style)
            if isinstance(val, float)
            else html.Td(val, style=cell_style)
            for val in dataframe.loc[idx]
        ]
        rows.append(html.Tr(row, style=cell_style))

    return html.Table(
        # Thead avec l'index puis les colonnes
        [html.Thead(html.Tr(header))] + [html.Tbody(rows)],
        style={
            "border": border,
            "border-collapse": "collapse",
            "margin-top": "20px",
            "font-size": 15,
            "color": mesPolices["color"],
            "font-family": mesPolices["font-family"],
            "font-weight": mesPolices["font-weight"],
        },
    )

In [19]:
# Définition du layout de l'application
app.layout = html.Div(
    [
        dcc.Graph(
            id="price",
            figure=price,
            config={"responsive": True},
        ),
        dcc.Graph(
            id="size",
            figure=size,
            config={"responsive": True},
        ),
        html.Div(
            id="table-container",
            children=generate_table(
                train[
                    [
                        "SalePrice",
                        "1stFlrSF",
                        "2ndFlrSF",
                        "TotalBsmtSF",
                        "KitchenQual",
                        "ExterQual",
                        "Fireplaces",
                        "Neighborhood",
                        "MiscFeature",
                        "OverallQual",
                        "FullBath",
                    ]
                ]
                .sort_values("SalePrice", ascending=False)
                .head(3)
            ),
            style={
                "overflowX": "auto",  # Active le scroll horizontal
                "width": "100%",  # Vous pouvez ajuster la largeur si nécessaire
                "padding": "10px",  # Optionnel : ajouter un peu d'espace
            },
        ),
        dcc.Graph(
            id="yr",
            figure=yr,
            config={"responsive": True},
        ),
        dcc.Graph(
            id="summer",
            figure=summer,
            config={"responsive": True},
        ),
    ],
    style={
        # "display": "flex",
        "flex-direction": "column",
        "align-items": "center",
    },
)

##### Affichage de l'app Dash


In [20]:
app.run(jupyter_mode="external", debug=True)

Dash app running on http://127.0.0.1:8050/
